In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow

In [41]:
df=pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ELP/train.csv', on_bad_lines='skip')
df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [42]:
df.shape

(6067, 8)

In [51]:
X = list(df['full_text'])

In [52]:
y = df.iloc[:,2:]

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [37]:
y_train

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
109,3.5,3.0,3.5,4.0,3.5,3.5
4248,2.5,3.0,2.5,2.0,2.0,2.0
5385,4.0,3.0,3.5,3.5,3.5,4.0
5087,3.5,3.5,3.5,3.5,4.0,4.5
2828,2.5,2.0,3.0,2.5,2.0,2.0
...,...,...,...,...,...,...
3772,3.0,3.0,3.0,3.5,3.5,3.0
5191,3.0,3.0,3.5,3.5,3.0,4.0
5226,3.0,3.0,4.0,3.5,3.0,3.5
5390,4.0,3.5,4.5,4.0,3.5,4.0


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.0 MB/s eta 0:00:00


In [31]:
from transformers import GPT2Tokenizer, TFGPT2Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [32]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2Model.from_pretrained('gpt2', output_hidden_states=True)

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [33]:
# train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors='tf')
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Using pad_token, but it is not set yet.


In [54]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)

In [55]:
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [56]:
import tensorflow as tf

In [58]:
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings), y_train
# ))

In [59]:
# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_encodings), y_test
# ))

In [ ]:
# if train_encodings.shape[1] > model.input_shape[-1]:
#     input_ids = train_encodings[:, :model.input_shape[-1]]

In [ ]:
# hidden_states_train = model(train_encodings['input_ids'], attention_mask=train_encodings['attention_mask'])[2][-2]
# hidden_states_test = model(test_encodings['input_ids'], attention_mask=test_encodings['attention_mask'])[2][-2]


In [ ]:
# hidden_states_train = Flatten()(hidden_states_train)
# hidden_states_test = Flatten()(hidden_states_test)

In [60]:
input_ids = tokenizer.encode(input_sentence, truncation=True, padding=True, return_tensors='tf')

# Ensure the input IDs are of the correct shape
if input_ids.shape[1] > model.input_shape[-1]:
    input_ids = input_ids[:, :model.input_shape[-1]]

NameError: ignored

In [ ]:
from transformers import TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [ ]:

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)